In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/episode-choose-your-story/Episodes__Choose_Your_Story_app_store_IN.csv
/kaggle/input/episode-choose-your-story/Episodes__Choose_Your_Story_play_store.csv
/kaggle/input/episode-choose-your-story/Episodes__Choose_Your_Story_app_store_US.csv
/kaggle/input/chapters/Chapters_Interactive_Stories_app_store_US.csv
/kaggle/input/chapters/Chapters_Interactive_Stories_play_store.csv
/kaggle/input/chapters/Chapters_Interactive_Stories_app_store_IN.csv
/kaggle/input/romance-club-stories-i-play/Romance_Club_Stories_I_Play_app_store_IN.csv
/kaggle/input/romance-club-stories-i-play/Romance_Club_Stories_I_Play_play_store.csv
/kaggle/input/romance-club-stories-i-play/Romance_Club_Stories_I_Play_app_store_US.csv
/kaggle/input/eva-character/EVA_Character_AI_Friend_play_store.csv
/kaggle/input/eva-character/EVA_Character_AI_Friend_app_store_IN.csv
/kaggle/input/eva-character/EVA_Character_AI_Friend_app_store_US.csv
/kaggle/input/romance-fate-story-and-chapters/Romance_Fate_Story_and_Chapters_

In [2]:
!pip install spello

In [3]:
!pip install demoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.8 MB/s eta 0:00:00


In [4]:
!pip install googletrans

In [5]:
# !pip install contractions

In [7]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords 
import re,string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
import emoji
import pandas as pd
import numpy as np
import nltk
import googletrans
# import contractions
from googletrans import Translator
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from nltk.corpus import wordnet
from sklearn.metrics import classification_report

In [8]:
nltk.download('omw-1.4')  # For WordNet lemmatizer
nltk.download('punkt_tab')    # For tokenization

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
import keras
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from string import ascii_lowercase
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from collections import Counter
from transformers import pipeline

2025-05-12 16:14:55.928327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747066496.180624      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747066496.251938      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
# Read the CSV file and display the first few rows
df1 = pd.read_csv('/kaggle/input/eva-character/EVA_Character_AI_Friend_play_store.csv')
df2 = pd.read_csv('/kaggle/input/eva-character/EVA_Character_AI_Friend_app_store_US.csv')
df2.rename(columns={"userName": "user_name"}, inplace=True)
df2.rename(columns={"date": "review_date"}, inplace=True)
df2.rename(columns={"developerResponse": "developer_response"}, inplace=True)
df2.rename(columns={"review": "review_description"}, inplace=True)
df = pd.concat([df2, df1], ignore_index=True)
# df = df.iloc[13000: 17000, : ]

In [13]:
df 

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,review_date,developer_response,review_description,rating,isEdited,title,user_name,review_id,userImage,thumbs_up,reviewCreatedVersion,developer_response_date,appVersion
0,2021-08-18 16:57:22,"{'id': 24624600, 'body': 'Hello! Thank you so ...",I would love to give this app 5 stars so I'll ...,4,False,I want to give 5 stars but...,hendihebd,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-27 05:04:51,"{'id': 35286199, 'body': 'Hey dear! Thank you ...",The Ai is fairly smart. I have tried others bu...,4,False,"Fun, sexy, but a little pricey",Toon Guy,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-29 16:12:31,"{'id': 24867565, 'body': 'Hello! Thank you for...",This app I got yesterday like last night and s...,4,False,Ai friend indeed does help a bit with loneliness,LONGSIDES,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-22 16:11:49,"{'id': 34483490, 'body': ""Hello! Thanks for yo...",So the first thing that I noticed that I didn’...,2,False,Disappointing Performance,KyOty the White,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-05-02 16:18:29,"{'id': 36369749, 'body': ""Hello! Thanks for yo...",In just the hour that I have been using this a...,4,False,Pretty good!,kstafford0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16084,2021-03-26 00:43:43,Hi! Thanks for your feedback. You can spin the...,A total deceptive app. It tells you you have t...,1,NaN,NaN,Tony Rowan,c3a741e7-7fdc-454b-b608-c0974036bff6,https://play-lh.googleusercontent.com/a/ACg8oc...,10.0,NaN,2021-05-08 18:45:13,NaN
16085,2021-03-25 06:44:21,Hi! Thank you for rating us! It is wonderful t...,Just installed the app. I should withhold my o...,5,NaN,NaN,Erick Hensz,97b99da7-ded5-4473-96fe-c17d7e1a498c,https://play-lh.googleusercontent.com/a-/ALV-U...,2.0,1.3.0,2021-06-01 01:51:14,1.3.0
16086,2021-03-20 11:05:19,Hi! Thanks for your feedback. You can spin the...,"no time to evaluate, about a minute, before yo...",1,NaN,NaN,Orion 123abc,b6265440-bafa-4b0d-a387-7de18766c14e,https://play-lh.googleusercontent.com/a/ACg8oc...,19.0,1.3.0,2021-05-08 18:28:56,1.3.0
16087,2021-03-09 23:14:25,Hello! Many thanks for your review. It's great...,good,5,NaN,NaN,gavin smith,3b34256b-2bfd-449c-9238-849e2253d764,https://play-lh.googleusercontent.com/a-/ALV-U...,1.0,1.1.1,2021-06-01 01:15:06,1.1.1


In [ ]:
# Drop rows where 'review_description' is NaN
df = df.dropna(subset=['review_description']) 
# Remove Empty Strings ('')
df = df[df['review_description'].str.strip() != '']
# Droping columns that are not needed
df.drop(columns=['review_date','title','userImage', 'user_name','thumbs_up', 'reviewCreatedVersion', 'appVersion', 'isEdited', 'developer_response_date','review_id'], inplace=True)
# expand to multiple word sequences (for example, “ain’t” → “am not”)
def expand(text):
    expanded = contractions.fix(text)
    return text
df['review_description'] = df['review_description'].apply(expand)

# text LowerCase
def text_lowercase(text):
    return text.lower()

df['review_description'] = df['review_description'].apply(text_lowercase)


# Remove punctuations
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
df['review_description'] = df['review_description'].apply(remove_punctuation)

# Normalize special apostrophes and other unicode quotes
def clean_text(text):
    text = text.replace('’', "'").replace('‘', "'").replace('“', '"').replace('”', '"')
    return text
df['review_description'] = df['review_description'].apply(clean_text)
# Text Classification

## XLM - ROBERTa

# Load language detection model
detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")
# Function to classify a single text
def classify_text(text):
    if pd.isna(text):  # Handle NaN values
        return None
    result = detector(text, top_k=1, truncation=True)[0]  # Get top result
    return result['label']  # Extract predicted language
# Apply function to the 'review_description' column
df['label'] = df['review_description'].apply(classify_text)
print('after applying ROBERTa: ',df['label'].value_counts())
## RNN
from tensorflow.keras.models import model_from_json, Sequential
# Load the model architecture from JSON file
with open("/kaggle/input/dataset/lang_class_relu_model.json", "r") as json_file:
    loaded_model_json = json_file.read()
# Reconstruct the model and provide the custom objects mapping
model_classifier = model_from_json(loaded_model_json, custom_objects={'Sequential': Sequential})
# Load the trained weights into the model
model_classifier.load_weights("/kaggle/input/dataset/lang_class_relu_model_weights.h5")
# Compile the model before using it
model_classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model loaded successfully!")
# Create character encoding dictionary
od = {ch: idx for idx, ch in enumerate(ascii_lowercase, 1)}

# Function to preprocess a word (convert to numerical sequence)
def preprocess_word(word):
    word = word.lower()  # Convert to lowercase
    word = re.sub(r"[^a-z]", "", word)  # Remove non-alphabetic characters
    seq = [od.get(char, 0) for char in word]  # Convert to numbers
    return seq

# Function to predict language for a single word
def predict_word_language(word, model):
    seq = preprocess_word(word)
    if not seq:  # If word is empty after cleaning, return "en" by default
        return "en"

    seq = [seq]  # Convert to list of lists for padding
    seq_padded = pad_sequences(seq, maxlen=10)  # Pad with a smaller maxlen
    seq_padded = np.array(seq_padded).reshape(1, 10, 1)  # Reshape for LSTM

    # Predict
    prediction = model.predict(seq_padded)

    return "hi" if prediction[0][0] > 0.5 else "en"

# Function to predict language for a full sentence
def predict_language(text, model):
    words = text.split()  # Split sentence into words
    predictions = [predict_word_language(word, model) for word in words]

    # Count occurrences of each label
    counter = Counter(predictions)
    return counter.most_common(1)[0][0]  # Return most frequent label

# Remove Empty String
df = df[df['review_description'].str.strip() != '']

# Apply the function
df1 = df[df['label'] != 'en']
print('df1 shape:', df1.shape)
df1['label'] = df1['review_description'].apply(lambda x: predict_language(x, model_classifier))
print('after applying RNN: ',df1['label'].value_counts())
df.update(df1)
print('now overall:  ',df['label'].value_counts())
# Filter the dataframe greater than 2 words
# Filter the dataframe to include only rows where the review description contains greater than 2 words.
df = df[
    df['review_description'].apply(lambda x: len(x.split()) > 2)
]

# Convert emojis into text
import demoji
demoji.download_codes()
def replace_emojis(text):
    emojis = demoji.findall(text)
    for emo, desc in emojis.items():
        text = text.replace(emo, f":{desc}:")
    return text

df['review_description'] = df['review_description'].apply(replace_emojis)
Translation
df2 = df[df['label'] != 'en']

import google.generativeai as genai
import pandas as pd
import time

# Configure the Generative AI API
genai.configure(api_key="Your API key")

def extract_text(response):
    """Extracts and concatenates text parts from the response candidate."""
    if not response.candidates:
        return ""
    candidate = response.candidates[0]
    if not candidate.content.parts:
        return ""
    return "".join([part.text for part in candidate.content.parts])

def translate_and_correct_grammar(text):
    model = genai.GenerativeModel('gemini-1.5-flash-001')
    
    # Step 1: Translate from Hindi to English
    translate_prompt = f"Translate the following text to English ,just return the translated text and nothing else:\n\n{text}"
    translated_response = model.generate_content(translate_prompt)
    translated_text = extract_text(translated_response)
    
    # Step 2: Correct grammar of the translated text
    correct_prompt = f"Rewrite the following sentence and fix any grammar issues and if no grammer issues are there then just write the sentenece don't give any explaination:\n\n{translated_text}"
    corrected_response = model.generate_content(correct_prompt)
    corrected_text = extract_text(corrected_response)
    
    return translated_text

def process_review(row):
    return translate_and_correct_grammar(row['review_description'])

df_new = pd.DataFrame()

# Process reviews in batches of 6 to avoid rate limiting
for start_index in range(0, len(df2), 6):
    time.sleep(55)  # Sleep for 55 seconds to avoid rate limiting
    df_next_6 = df2.iloc[start_index:start_index+7].copy()
    
    # Apply the function to each row (gets directly the corrected text)
    df_next_6['review_description'] = df_next_6.apply(process_review, axis=1)
    df_new = pd.concat([df_new, df_next_6], ignore_index=True)

# df.update(df_new)
df.drop(df[df['label'] != 'en'].index, inplace=True)
df.drop(columns=['label'], inplace=True)
from spello.model import SpellCorrectionModel

# Load the model
sp = SpellCorrectionModel(language='en')
sp.load('/kaggle/input/dataset/en_large.pkl')

# Apply spell correction to each review (if it's a string)
def correct_spelling(x):
    if isinstance(x, str):
        return sp.spell_correct(x)['spell_corrected_text']
    return x  # Return as-is if it's not a string

df['review_description'] = df['review_description'].apply(correct_spelling)
# Lemmatization using NLTK
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
# Create a WordNetLemmatizer object 
lemmatizer = WordNetLemmatizer() 
df['review_description'] = df['review_description'].apply(
    lambda text: " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(text)])
)
# Remove stop words
# importing stop words from nltk corpus
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return " ".join(filtered_tokens)

df['review_description_after_removing_stop_words'] = df['review_description'].apply(remove_stopwords)

# Removing Numbers
pattern = r'[0-9]'
df['review_description'] = df['review_description'].apply(lambda x: re.sub(pattern, '', x))
df['review_description_after_removing_stop_words'] = df['review_description_after_removing_stop_words'].apply(lambda x: re.sub(pattern, '', x))

import emoji
# Remove Emojis
def remove_using_emoji(txt):
    return emoji.replace_emoji(txt, '')
df['review_description'] = df['review_description'].apply(remove_using_emoji)
df['review_description_after_removing_stop_words'] = df['review_description_after_removing_stop_words'].apply(remove_using_emoji)

# text LowerCase
def text_lowercase(text):
    return text.lower()
df['review_description'] = df['review_description'].apply(text_lowercase)
df['review_description_after_removing_stop_words'] = df['review_description_after_removing_stop_words'].apply(text_lowercase)

# Remove punctuations
def remove_punctuation(text):
        translator = str.maketrans('', '', string.punctuation)
        return text.translate(translator)
df['review_description'] = df['review_description'].apply(remove_punctuation)
df['review_description_after_removing_stop_words'] = df['review_description_after_removing_stop_words'].apply(remove_punctuation)

#Drop rows where 'review_description' is NaN
df= df.dropna(subset=['review_description']) 

#Remove Empty Strings ('')
df = df[df['review_description'].str.strip() != '']

# Filter dataframe again
# Filter the dataframe to include only rows where the review description contains greater than 2 words.
df = df[
    df['review_description'].apply(lambda x: len(x.split()) > 2)
]
df = df[
    df['review_description_after_removing_stop_words'].apply(lambda x: len(x.split()) > 2)
]


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
df

In [ ]:
df.to_csv('EVA_Character_AI_Friend_US.csv', index=False)